In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#Uses gpu, if thats not possible it makes use of the cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Resizes the images to 128x128 pixels, Turns them into a tensor, and normalizes them
tf = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

#Data Loaders(Training Data, Testing Data)
train_dl = DataLoader(
    datasets.ImageFolder('data/Training/',tf),
     batch_size=32, shuffle=True, num_workers=4, pin_memory=True
)

test_dl = DataLoader(
    datasets.ImageFolder('data/Testing/',tf),
    batch_size=32, shuffle=False, num_workers=4, pin_memory=True
)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(128*16*16, 256), nn.ReLU(), nn.Dropout(0.5),
    nn.Linear(256, 4) #4 classes
).to(device)

In [ ]:
#defines an Optimizer
opt = optim.AdamW(model.parameters(), 1e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#Training Loop

model.train() #Put model into training mode

#loop thru all the images. 25 times
for epoch in range(25):
    runnig_loss = 0 #keeps track of each loss during an iteration

    for x, y in train_dl: #x->inputs, y->labels
        opt.zero_grad() #clears gradients each iteration so they dont mix old gradients with new ones

        loss = loss_fn(model(x.to(device)), y.to(device)) #moves data to gpu or cpu,  makes a forward pass (model(x)), calculates the loss
        loss.backward() #calculates gradients(how much to change the weigh, up or down a bit)

        runnig_loss += loss #adds current loss to total loss for current epoch too see how bad the model did during that iteration

        opt.step() #updates models internal valus using our updated gradients so it can make fewer mistakes
    print(f'Epoch {epoch+1}: Loss was {runnig_loss}')

In [ ]:
#Evaluation Loop


model.eval() #Turns off training only behaviours


test_loss, correct = 0.0, 0 #keeps track of total incorrrect, and correct predictions

with torch.no_grad(): #Restricts the use of gradients, doesnt store info needed for backpropagation
    for x,y in test_dl: #loops thru test dataloader, and gives model data in batches
        x,y = x.to(device), y.to(device) #puts batchs into gpu or cpu depending on avaliblity

        logits = model(x) #does a forward pass (asks model fo its preditions)
        test_loss += loss_fn(logits,y).item() * y.size(0) #takes the loss of the patch multiplys it by the size of the batch to get the average loss of the batch and adds it to the total loss

        preds = logits.argmax(dim=1) #for each row in the batch it picks the index of the highest number (highest number = models prediction)
        correct += (preds == y).sum().item() #counts how much of the predcition were right 

test_loss /= len(test_dl.dataset) #takes the total loss and divdes it by the number of test sample to get the average loss per test
accuracy = 100.0 * correct/len(test_dl.dataset) #takes the total corect devided by the test sample and multiple by 100 to get the accuracy percentage

print('Test Loss:', test_loss, 'Test Accuracy', accuracy,'%') #prints total loss and accruacu

In [ ]:
import random

import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image

model.eval() #puts model in evaluation mode

indx = random.randrange(len(test_dl.dataset)) #picks a random number from 0 to the length of the test data set
img, label = test_dl.dataset[indx] #variables to store the image at the index found above, and its label

unnormalize = img * 0.5 + 0.5 #unnormalizes the image so it can be seen by humans (when normliazed the images pixel values are changed ot -1 and 1, but images cant be shown proberly in that range, so you cahnge the pixel values back to 0 and 1)
plt.imshow(to_pil_image(unnormalize)) #Converts pyTorch tensor to normal image and displays it
plt.axis('off') #hides x and y grid lines
plt.title('Sample from Test Set') #Give image a title

with torch.no_grad(): #Tells model to not track graidents as we are not training anymore(waste of memory)
    logits = model(img.unsqueeze(0).to(device)) #model does its evaltion on img and gets its raw scores for each class (how likley each type is to be true) (unsqueeze adds a new dimension so that it can be accepted [3,32,32] -> [1,3,32,32], creates a batch of size 1) 
    pred = logits.argmax(1).item() #gets the index of the models largest score(which is its prediction)

class_names = test_dl.dataset.classes #list of the class_names [the 4 brain tumor types]
print(f"Predicted Class: {class_names[pred]}") #Gets the name of the predicted brain tumor 
print(f"Ground Truth: {class_names[label]} ") #Gets the real name of the brain tumor
